## 1ª parte: 
## formulação do banco de dados de teses e dissertações sobre um tema específico

In [60]:
import requests
import json
import pandas as pd
from tqdm.auto import tqdm

# Termos da busca

url = 'http://catalogodeteses.capes.gov.br/catalogo-teses/rest/busca'

headers = {'Content-type': 'application/json','Accept': 'text/plain'}

ano = [{"campo": "Ano", "valor": "2004"},{"campo": "Ano", "valor": "2005"},{"campo": "Ano", "valor": "2006"},
       {"campo": "Ano", "valor": "2007"},{"campo": "Ano", "valor": "2008"},{"campo": "Ano", "valor": "2009"},
       {"campo": "Ano", "valor": "2010"},{"campo": "Ano", "valor": "2011"},{"campo": "Ano", "valor": "2012"},
       {"campo": "Ano", "valor": "2013"},{"campo": "Ano", "valor": "2014"},{"campo": "Ano", "valor": "2015"},
       {"campo": "Ano", "valor": "2016"},{"campo": "Ano", "valor": "2017"},{"campo": "Ano", "valor": "2018"},
       {"campo": "Ano", "valor": "2019"},{"campo": "Ano", "valor": "2020"},{"campo": "Ano", "valor": "2021"}]

termos = {'termo': '"escola sem partido" "projeto escola sem partido" "PL 7180/2014" "PL 7180/14" "PL7180/2014" "PL7180/14" "PL 867/2015" "PL 867/15" "PL867/15" "PL867/2015" "ideologia de gênero" "gender ideology" "doutrinação ideológica" "doutrinação" "doutrinação marxista" "comissão especial escola sem partido" "doutrinação esquerdista"',
          'registrosPorPágina': 20, 
          'filtros': ano}

# função que retorna os resultados da busca (método post)

def get(pp):
    busca = termos.copy()
    busca['pagina'] = pp

    while True:
        r = requests.post(url, headers=headers, data= None, json = busca)
        if r.status_code == 200:
            return r.json()
        else:
            continue
            
# retorna a quantidade de páginas do resultado

def pag(r):
    return round((r['total'] / r['registrosPorPagina'])+0.5) 

# resutados por página

total = 1
first = 1
resultado = []

while first <= total:
    produto = get(first) #pega todos os dados dos trabalhos por página
    
    if first == 1:
        total = pag(produto)

    for trabalho in produto['tesesDissertacoes']:  # cria um json com infos limpas de cada item
        item = trabalho
        resultado.append(item)
    
    first += 1
    print('Total de casos: ', len(resultado))

Total de casos:  20
Total de casos:  40
Total de casos:  60
Total de casos:  80
Total de casos:  100
Total de casos:  120
Total de casos:  140
Total de casos:  160
Total de casos:  180
Total de casos:  200
Total de casos:  220
Total de casos:  221


In [61]:
#visão do json() de retorno, que descreve o tipo de informação.
resultado[0]

{'id': '5303932',
 'instituicao': 'UNIVERSIDADE FEDERAL DE GOIÁS',
 'nomePrograma': 'Direitos Humanos',
 'municipioPrograma': 'Goiânia',
 'titulo': 'As identidades de gênero e sexualidade na visão dos parlamentares da Câmara Federal: uma análise do discurso a partir dos projetos "escola sem partido"',
 'autor': 'ISABELLA BRUNA LEMES PEREIRA',
 'dataDefesa': '2017-08-17',
 'volumes': '1',
 'paginas': '242',
 'biblioteca': 'http://repositorio.bc.ufg.br/tede/handle/tede/8065',
 'grauAcademico': 'Mestrado',
 'link': 'https://sucupira.capes.gov.br/sucupira/public/consultas/coleta/trabalhoConclusao/viewTrabalhoConclusao.jsf?popup=true&id_trabalho=5303932'}

## Salvar os dados gerais em um dataframe para posterior análise.

In [64]:
# nome das colunas
colunas = ['id',
 'instituicao',
 'nomePrograma',
 'municipioPrograma',
 'titulo',
 'autor',
 'dataDefesa',
 'volumes',
 'paginas',
 'biblioteca',
 'grauAcademico',
 'link']

In [65]:
# cria dataframe
dados = pd.DataFrame(resultado, columns=colunas)
# salva banco csv 
resultado = pd.DataFrame(dados)
resultado.to_csv('TesesDissertacoes_ESP_Geral.csv')

## Importando os dados do dataframe para operações.

In [66]:
arquivo = (r'TesesDissertacoes_ESP_Geral.csv')
suc = pd.read_csv(arquivo ,header=0, sep=',')

In [67]:
#Quantos casos existem no banco?
len(suc)

221

In [68]:
suc.head(2)

,Unnamed: 0,id,instituicao,nomePrograma,municipioPrograma,titulo,autor,dataDefesa,volumes,paginas,biblioteca,grauAcademico,link
0,0,5303932,UNIVERSIDADE FEDERAL DE GOIÁS,Direitos Humanos,Goiânia,"As identidades de gênero e sexualidade na visão dos parlamentares da Câmara Federal: uma análise do discurso a partir dos projetos ""escola sem partido""",ISABELLA BRUNA LEMES PEREIRA,2017-08-17,1,242,http://repositorio.bc.ufg.br/tede/handle/tede/8065,Mestrado,https://sucupira.capes.gov.br/sucupira/public/consultas/coleta/trabalhoConclusao/viewTrabalhoConclusao.jsf?popup=true&id_trabalho=5303932
1,1,6452576,PONTIFÍCIA UNIVERSIDADE CATÓLICA DO RIO DE JANEIRO,CIÊNCIAS SOCIAIS,Rio de Janeiro,Escola pública sob disputa: moralidade e religião,SARAH LAURINDO MONTEIRO,2018-04-20,1,180,Biblioteca Central da PUC-Rio,Mestrado,https://sucupira.capes.gov.br/sucupira/public/consultas/coleta/trabalhoConclusao/viewTrabalhoConclusao.jsf?popup=true&id_trabalho=6452576


In [70]:
#Quantos trabalhos possuem links para o resumo?
suc['link'].count()

171

In [71]:
# Como uma parte das pesquisas são anteriores a criação da plataforma, a coluna 'link' descreve os casos faltantes.
suc.isnull().sum()

Unnamed: 0           0 
id                   0 
instituicao          0 
nomePrograma         0 
municipioPrograma    0 
titulo               0 
autor                0 
dataDefesa           0 
volumes              0 
paginas              0 
biblioteca           0 
grauAcademico        0 
link                 50
dtype: int64

In [72]:
#cria dataframe com os links para extrair o resumo que não retornou com json
links = suc['link']

In [76]:
#Segunda precisa mais formas. Transforma o resultado em um objeto de série Pandas.
pd.set_option('display.max_colwidth', -1)
import numpy as np
links2 = links.dropna(axis=0, how='all')


<ipython-input-76-744e54fd7b6f>:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [78]:
## Salvar os dados dos links
links2.to_csv('links_TesesCapes_ESP.csv', encoding='CP1252')